In [1]:
import pandas as pd
import numpy as np
import re
from itertools import product
from toolz.dicttoolz import merge_with
from aocd import get_data

In [2]:
data = get_data(year=2018, day=4)

# Part 1

In [5]:
def parse(x):
    return re.match('\[(.*)\] (.*)', x).groups()

In [10]:
df = pd.DataFrame([*map(parse, data.splitlines())], columns=['Date', 'Action']).assign(Date=lambda d: pd.to_datetime(d.Date.str.replace('15(\d\d)', r'20\1'))).sort_values('Date')

In [17]:
df['Guard'] = df.Action.str.extract('Guard #(\d+)', expand=False).ffill()
df['Action'] = df.Action.str.replace('Guard #(.*) ', '')

In [38]:
df = df.set_index('Date').asfreq('T').ffill().query('Date.dt.hour == 0').assign(Asleep=lambda d: d.Action.map(lambda x: 1 if x == 'falls asleep' else 0))

In [45]:
d_ = df.set_index([df.index.date, 'Guard', df.index.minute]).Asleep.rename_axis(['Date', 'Guard', 'Time']).unstack(fill_value=0)

In [50]:
d_.sum(1).groupby('Guard').sum().idxmax()

'1571'

In [54]:
d_.xs('1571', level=1).sum().idxmax()

54

In [55]:
54 * 1571

84834

# Part 2

In [63]:
d_.groupby('Guard').sum().max(1).idxmax()

'1619'

In [66]:
d_.groupby('Guard').sum().loc['1619'].idxmax()

33

In [67]:
1619 * 33

53427

In [70]:
from parse import parse
import pandas as pd
from collections import defaultdict

gtmp = "[{t:ti}] Guard #{i:d} begins shift"
etmp = "[{t:ti}] {e}"

def p(d):
  r = parse(gtmp, d)
  if r is None:
    r = parse(etmp, d)

  if r is not None:
    return r.named

records = [*map(p, sorted(data.splitlines()))]

In [71]:
i = None
guards = defaultdict(int)
minute = defaultdict(lambda:defaultdict(int))

s = 0
for rec in records:
  if 'i' in rec:
    i = rec['i']
  else:
    m = rec['t'].minute
    if rec['e'] == 'falls asleep':
      s = m
    else:
      while s < m:
        guards[i] += 1
        minute[i][s] += 1
        s += 1

In [72]:
g = max(guards, key=guards.get)
g * max(minute[g], key=minute[g].get)

84834

In [73]:
g = max(minute, key=lambda x: max(minute[x].values()))
g * max(minute[g], key=minute[g].get)

53427